In [ ]:
# Used to just get the size and failure rate of a dataset

from SimDataScraper import _GatherSims
DIR = "..\\ThesisDataSims\\SimFiles\\Saves\\Interpolation\\"
SID_KEYS, FR = _GatherSims(DIR)
print("Failure Rate: %0.2f%% for %d sims" % (FR * 100.0, len(SID_KEYS)))

In [ ]:
# This script removes thermal log files missing measurements. There may have been a bug with LTSpice on Windows 11, but some thermal simulations completed but didn't write the measurements
# So this script was made to remove those bugged cases

import os
from SimDataScraper import _GatherSims

DIR = "..\\ThesisDataSims\\SimFiles\\Saves\\Interpolation\\"
SID_KEYS, _= _GatherSims(DIR)

count = 1
for SID in list(SID_KEYS):
    
    for i in range(2):
        
        titlePrefix = "THERM_" + ("T" if not i else "B")
        LOG_FILE = DIR  + titlePrefix + "_" + SID + ".log"
        thermWriteSuccess = False

        with open(LOG_FILE , 'r') as file:
            line = file.readline()
            while line != "":
                if line.startswith("Measurement:"):  # Look to see if the thermal log file has the measurement line
                    thermWriteSuccess = True
                    break

                line = file.readline()

        if not thermWriteSuccess:
            print(LOG_FILE)
            os.remove(LOG_FILE)

    count += 1

In [ ]:
# Actually creating the database which will invoke other functions to gather the data.

from SimDataScraper import *
#Desktop\ThesisDataSims\SimFiles\Saves
dataset = SimulationDataset("..\\ThesisDataSims\\SimFiles\\Saves\\Interpolation\\", 
                            None, 
                            None, 
                            transform=MinMaxScale())
                            

In [ ]:
# Getting the slew rates and packing into a dictonary

from SimDataScraper import _GatherSims
from SimDataScraper import _GenerateSlewRate
from SimDataScraper import EngNotationToFloat
import pickle


DIR = "..\\ThesisDataSims\\SimFiles\\Saves\\Interpolation\\"
SID_KEYS,_ = _GatherSims(DIR)

with open(".\\InterpolationDumpFile", 'rb+') as datasetFile:
    dataset = pickle.load(datasetFile)


# Each SID is based on a random circuit configuration, independent of the case temperature. The case temperature is swept through ten points meaning each SID is really ten simulation points.
# The problem is that one one SID is stored for every ten of these points. This block below simple repeats a SID ten times for every case temperature point. 
# This is something that should be fixed in the SimDataScraper.py file
SIDLIST = []
for SID in dataset.rawDict["SID"]:
    SIDLIST.extend([SID] * 10)

dataset.rawDict["SID"] = SIDLIST
dataset.SimDict["SID"] = SIDLIST
del SIDLIST

datasetDict = {}
for i, x in enumerate(dataset.rawDict["SID"]):
    datasetDict[x] = dataset.rawDict["F"][i]
del dataset
    



slewDict = {}
slewDict["SID"] = SID_KEYS

count = 1
finalCount = len(SID_KEYS)
for SID in SID_KEYS:
    print("(%d/%d):%s" % (count, finalCount, SID))
    freq = datasetDict[SID]
    tempDict = {}

    # Vd slew
    tempDict |= _GenerateSlewRate(DIR, SID, freq, "v(vd1,vp)", "v_ds_t")
    tempDict |= _GenerateSlewRate(DIR, SID, freq, "v(vd2,vss)", "v_ds_b")
    # Id slew
    tempDict |= _GenerateSlewRate(DIR, SID, freq, "i(id_meas1)", "id_t")
    tempDict |= _GenerateSlewRate(DIR, SID, freq, "i(id_meas2)", "id_b")
    # Vg Slew
    tempDict |= _GenerateSlewRate(DIR, SID, freq, "v(vg1,vp)", "v_gs_t")
    tempDict |= _GenerateSlewRate(DIR, SID, freq, "v(vg2,vss)", "v_gs_b")
    # Ig slew
    tempDict |= _GenerateSlewRate(DIR, SID, freq, "i(Rg1)", "ig_t")
    tempDict |= _GenerateSlewRate(DIR, SID, freq, "i(Rg2)", "ig_b")

    for key in tempDict:
        slewDict.setdefault(key, []).append(tempDict[key])

    count += 1

In [ ]:
# Getting Vdson by masking out the time a switch is on based on the known frequency and some buffer to avoid ringing signals in the measurements

from SimDataScraper import _GatherSims
from SimDataScraper import _GenerateSlewRate
from SimDataScraper import EngNotationToFloat
import ltspice
import pickle
import numpy as np

def SearchFloat(data, value):
    diff = np.array(data - ([value] * len(data)))
    return np.argmin(np.absolute(diff))

DIR = "..\\ThesisDataSims\\SimFiles\\Saves\\Interpolation"
SID_KEYS,_ = _GatherSims(DIR)

with open(".\\InterpolationDumpFile", 'rb+') as datasetFile:
    dataset = pickle.load(datasetFile)


# Each SID is based on a random circuit configuration, independent of the case temperature. The case temperature is swept through ten points meaning each SID is really ten simulation points.
# The problem is that one one SID is stored for every ten of these points. This block below simple repeats a SID ten times for every case temperature point. 
# This is something that should be fixed in the SimDataScraper.py file
SIDLIST = []
for SID in dataset.rawDict["SID"]:
    SIDLIST.extend([SID] * 10)

dataset.rawDict["SID"] = SIDLIST
dataset.SimDict["SID"] = SIDLIST
del SIDLIST

datasetDict = {}
for i, x in enumerate(dataset.rawDict["SID"]):
    datasetDict[x] = dataset.rawDict["F"][i]
del dataset
    

vdsonDict = {}
vdsonDict["SID"] = SID_KEYS

count = 1
finalCount = len(SID_KEYS)
for SID in SID_KEYS:
    print("(%d/%d):%s" % (count, finalCount, SID))
    freq = datasetDict[SID]

    lts = ltspice.Ltspice(DIR +"\\SIM_" + SID + ".raw")
    lts.parse()
    time = lts.get_time()
    vds = lts.get_data("v(vd1,vp)")

    maxList = []
    minList = []
    midList = []
    # For each measured cycle
    for i in range(3):
        cycStart = 1.55 + i
        cycEnd = 1.95 + i
        cycMid = 1.75 + i

        start = SearchFloat(time, (cycStart/freq))
        end = SearchFloat(time, (cycEnd/freq))
        mid = SearchFloat(time, (cycMid/freq))

        vdsondata = vds[start:end]
        maxList.append(np.max(vdsondata))
        minList.append(np.min(vdsondata))
        midList.append(vds[mid])
    
    vdsonDict.setdefault("v_ds_on_max_t", []).append(np.max(maxList))
    vdsonDict.setdefault("v_ds_on_min_t", []).append(np.min(minList))
    vdsonDict.setdefault("v_ds_on_pp_t", []).append(np.max(maxList) - np.min(minList))
    vdsonDict.setdefault("v_ds_on_mid_t", []).append(np.mean(midList))

    vds = lts.get_data("v(vd2,vss)")
    maxList = []
    minList = []
    midList = []
    for i in range(3):
        cycStart = 1.55 - 0.5 + i
        cycEnd = 1.95 - 0.5 + i
        cycMid = 1.75 - 0.5 + i
        start = SearchFloat(time, (cycStart/freq))
        end = SearchFloat(time, (cycEnd/freq))
        mid = SearchFloat(time, (cycMid/freq))

        vdsondata = vds[start:end]
        maxList.append(np.max(vdsondata))
        minList.append(np.min(vdsondata))
        midList.append(vds[mid])
    
    vdsonDict.setdefault("v_ds_on_max_b", []).append(np.max(maxList))
    vdsonDict.setdefault("v_ds_on_min_b", []).append(np.min(minList))
    vdsonDict.setdefault("v_ds_on_pp_b", []).append(np.max(maxList) - np.min(minList))
    vdsonDict.setdefault("v_ds_on_mid_b", []).append(np.mean(midList))

    count += 1

In [ ]:
# Using the pickle library to seralize (write to a file) the dataset to make it easy to transfer
# Can be modified to save the Vdson and Slew rate structures


import pickle

fileName = ".\\InterpolationDumpFile"

with open(fileName, 'wb+') as dataFile:
    pickle.dump(dataset, dataFile)

In [ ]:
# Checks that the seralized file was successful by loading it and checking a few keys

import pickle

fileName = ".\\InterpolationDumpFile"
with open(fileName, 'rb+') as dataFile:
    readDataset = pickle.load(dataFile)

#print(len(readDataset.rawDict["SID"]))
#print(len(readDataset.rawDict["F"]))
#print(len(readDataset.rawDict["Tc"]))
#print(len(readDataset.rawDict["Tj_Final_T"]))
#print(len(readDataset.rawDict["Tj_Final_B"]))

for key in readDataset:
    print(key, end=": ")
    print(readDataset[key][:10])

del readDataset